## Zadanie domowe - Algorytm Canny'ego

Celem zadania domowego jest wykonanie pełnej implementacji algorytmu Canny'ego.

W ramach ćwiczenia w trakcie laboratorium wyznaczono obrazy $g_{NH}$ i $g_{NL}$.
Dla przypomnienia:
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
Często krawędzie "pewne" nie są ciągłe.
Wykorzystuje się więc krawędzie "potencjalne", aby uzupełnić nieciągłości.
Procedura wygląda następująco:
1. Stwórz stos zawierający wszystkie piksele zaznaczone na obrazie $g_{NH}$.
W tym celu wykorzystaj listę współrzędnych `[row, col]`.
Do pobrania elementu z początku służy metoda `list.pop()`.
Do dodania elementu na koniec listy służy metoda `list.append(new)`.
2. Stwórz obraz, który będzie zawierał informację czy dany piksel został już odwiedzony.
3. Stwórz obraz, który zawierać będzie wynikowe krawędzie.
Jej rozmiar jest równy rozmiarowi obrazu.
4. Wykonaj pętlę, która będzie pobierać elementy z listy, dopóki ta nie będzie pusta.
W tym celu najlepiej sprawdzi się pętla `while`.
    - W każdej iteracji pobierz element ze stosu.
    - Sprawdź, czy dany element został już odwiedzony.
    - Jeśli nie został, to:
        - Oznacz go jako odwiedzony,
        - Oznacz piksel jako krawędź w wyniku,
        - Sprawdź otoczenie piksela w obrazie $g_{NL}$,
        - Dodaj do stosu współrzędne otoczenia, które zawierają krawędź.
        Można to wykonać np. pętlą po stworzonym otoczeniu.
7. Wyświetl obraz oryginalny, obraz $g_{NH}$ oraz obraz wynikowy.
8. Porównaj wynik algorytmu z wynikiem OpenCV.

Pomocnicze obrazy $g_{NH}$ i $g_{NL}$ zostały wprowadzone dla uproszczenia opisu.
Algorytm można zaimplementować w bardziej "zwarty" sposób.

Na podstawie powyższego opisu zaimplementuj pełny algorytm Canny'ego.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

def nomax(alpha, magnitude):
    x, y = alpha.shape
    maximums = np.zeros((x, y))

    for i in range(1, x - 1):
        for j in range(1, y - 1):
            if (-22.5 < alpha[i, j] < 22.5) or (-157.5 > alpha[i, j] > 157.5):
                if not (magnitude[i, j + 1] > magnitude[i, j] or magnitude[i, j - 1] > magnitude[i, j]):
                    maximums[i, j] = magnitude[i, j]

            elif (67.5 < alpha[i, j] < 112.5) or (-112.5 < alpha[i, j] < -67.5):
                if not (magnitude[i + 1, j] > magnitude[i, j] or magnitude[i - 1, j] > magnitude[i, j]):
                    maximums[i,j] = magnitude[i,j]

            elif (22.5 < alpha[i, j] < 67.5) or (-157.5 < alpha[i, j] < -112.5):
                if not (magnitude[i + 1, j + 1] > magnitude[i, j] or magnitude[i - 1, j - 1] > magnitude[i, j]):
                    maximums[i, j] = magnitude[i, j]

            elif (112.5 < alpha[i, j] < 157.5) or (-67.5 < alpha[i, j] < -22.5):
                if not (magnitude[i - 1, j + 1] > magnitude[i, j] or magnitude[i + 1, j - 1] > magnitude[i, j]):
                    maximums[i, j] = magnitude[i, j]

    return maximums

def canny(image, TL=60, TH=120, window_size = 3, sigma = 1):
    sobel_horizontal = np.array([[1,2,1],
                                [0,0,0],
                                [-1,-2,-1]])
    sobel_vertical = np.array([[-1, 0, 1],
                                [-2, 0, 2],
                                [-1, 0, 1]])

    image_gauss = cv2.GaussianBlur(image, (window_size, window_size), sigma)

    gx = cv2.filter2D(image_gauss, -1, sobel_vertical)
    gy = cv2.filter2D(image_gauss, -1, sobel_horizontal)

    magnitude = np.hypot(gx, gy)
    magnitude = magnitude / magnitude.max() * 255

    alpha = np.rad2deg(np.arctan2(gy, gx))

    maximums = nomax(alpha, magnitude)

    gN = maximums
    gNH = gN >= TH
    gNL = (TH > gN) & (gN >= TL)

    X,Y = image.shape

    coordinates_list =[]

    for i in range(1, X - 1):
        for j in range(1, Y - 1):
            if gNH[i, j] == 1:
                coordinates_list.append([i,j])

    visited = np.zeros(image.shape)
    edges  = np.zeros(image.shape)

    while coordinates_list:
        element = coordinates_list.pop()
        x = element[0]
        y = element[1]

        if visited[x,y] == 0:
            visited[x,y] = 1
            edges[x,y] = 1

            for k in range(x-1,x+2):
                for v in range(y-1,y+2):
                    if gNL[k, v] == 1:
                        coordinates_list.append([k,v])
    return edges

if not os.path.exists("dom.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/09_Canny/dom.png --no-check-certificate

I_dom = cv2.imread('dom.png', cv2.IMREAD_GRAYSCALE)

plt.imshow(canny(I_dom, 40, 120, 3, 1))
plt.axis('off')
plt.show()